In [1]:
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice, LocalWebservice
from azureml.core.model import Model, InferenceConfig
from azureml.core.environment import Environment, CondaDependencies
from azureml.core import Workspace


ws = Workspace.from_config()

Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto with exception (azureml-telemetry 1.6.0 (c:\projects\repos\workshops\env\lib\site-packages), Requirement.parse('azureml-telemetry~=1.2.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (azureml-core 1.6.0.post1 (c:\projects\repos\workshops\env\lib\site-packages), Requirement.parse('azureml-core~=1.2.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.ReusedStepRun = azureml.pipeline.core.run:StepRun._from_reused_dto with exception (azureml-core 1.6.0.post1 (c:\projects\repos\workshops\env\lib\site-packages), Requirement.parse('azureml-core~=1.2.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.StepRun = azureml.pipeline.core.run:StepRun._from_dto with exc

# Deploy to remote ACI

In [ ]:
myenv = Environment(name='pytorch-env')
myenv.python.conda_dependencies = CondaDependencies.create(
    pip_packages=['azureml-defaults', 'torch', 'torchvision>=0.5.0']
    )

inference_config = InferenceConfig(entry_script="../src/infer.py", environment=myenv)

aciconfig = AciWebservice.deploy_configuration(
    cpu_cores=3,
    memory_gb=1,
    tags={'data': 'cells-malaria',  'method':'transfer learning', 'framework':'pytorch'},
    description='Classify infected/uninfected cells using transfer learning with PyTorch'
    )
try:
    service = Webservice(ws,service_name)
    service.delete()
except Exception as e:
    print('Webservice does not exist yet')

model = Model(ws,name="pytorch-malaria-cell-class")

service = Model.deploy(
    workspace=ws,
    name=aci-cells-malaria
    models=[model],
    inference_config=inference_config,
    deployment_config=aciconfig,
    )
service.wait_for_deployment(show_output=True)
print(service.state)

# Deploy locally

In [ ]:
myenv = Environment(name='pytorch-env')
myenv.python.conda_dependencies = CondaDependencies.create(
    pip_packages=['azureml-defaults', 'torch', 'torchvision>=0.5.0']
    )

# explicitly set base_image to None when setting base_dockerfile
myenv.docker.base_image = None
myenv.docker.base_dockerfile = "FROM mcr.microsoft.com/azureml/base:intelmpi2018.3-ubuntu16.04"
myenv.inferencing_stack_version = "latest"

inference_config = InferenceConfig(entry_script="../src/infer.py", environment=myenv)

aciconfig = AciWebservice.deploy_configuration(
    cpu_cores=3,
    memory_gb=1,
    tags={'data': 'cells-malaria',  'method':'transfer learning', 'framework':'pytorch'},
    description='Classify infected/uninfected cells using transfer learning with PyTorch'
    )
localconfig = LocalWebservice.deploy_configuration(port=6789)
deployment_config = localconfig
service_name = 'local'
try:
    service = Webservice(ws,service_name)
    service.delete()
except Exception as e:
    print('Webservice does not exist yet')

model = Model(ws,name="pytorch-malaria-cell-class")

service = Model.deploy(
    workspace=ws,
    name=service_name,#aci-cells-malaria
    models=[model],
    inference_config=inference_config,
    deployment_config=deployment_config,
    )
service.wait_for_deployment(show_output=True)
print(service.state)